In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

In [ ]:
# Start with Einstein Telescope

In [ ]:
et_curve=np.genfromtxt("ET-0000A-18_ETDSensitivityCurveTxtFile.txt", 
                       names=('freq', 'ET-D-HF', 'ET-D-LF', 'Sum'))

In [ ]:
et_curve = Table(et_curve)

In [ ]:
ce_curve = np.genfromtxt("ce_wb.dat", names=('freq','CE'))

In [ ]:
ce_curve = Table(ce_curve)

In [ ]:
def Sn2OmGW_wT(freq, s_of_n, time):
    """
    Set H_0=68 km/s/Mpc
    1 pc = 1.029e+8 light seconds
    int_t_improv: integration time improves the detector sensitivity by sqrt(time*freq)
    """
    pc2sec = 1.029e+8
    metre2sec = 3.3356e-9
    H0_secs = 68*1000*metre2sec/(1e6*pc2sec) # H should be 100!!
    num = 2*np.pi**2*freq**3*s_of_n**2 # quantity in files is really sqrt(S_n(f))
    deno = 3*H0_secs**2
    int_t_improv = np.sqrt(time*freq)
    OmegaGW = num/deno/int_t_improv
    return OmegaGW

def Sn2OmGW(freq, s_of_n):
    """
    Set h=1 and use H_0=100 km/s/Mpc
    1 pc = 1.029e+8 light seconds
    """
    pc2sec = 1.029e+8
    metre2sec = 3.3356e-9
    H0_secs = 68*1000*metre2sec/(1e6*pc2sec)
    num = 2*np.pi**2*freq**3*s_of_n**2 # quantity in files is really sqrt(S_n(f))
    deno = 3*H0_secs**2
    OmegaGW = num/deno
    return OmegaGW

In [ ]:
plt.loglog(ce_curve['freq'], Sn2OmGW_wT(ce_curve['freq'], ce_curve['CE'], time2y), label="CE, 2 yrs")
plt.loglog(ce_curve['freq'], Sn2OmGW_wT(ce_curve['freq'], ce_curve['CE'], time5y), label="CE, 5 yrs")



plt.grid(True)
plt.ylabel("$\Omega_{GW}(f)$")
plt.xlabel("frequency (Hz)")
plt.legend()
plt.tight_layout()
#plt.savefig("ET_om_gw_2yr_integration.pdf")
plt.show()

In [ ]:
time2y = 2*3.154e+7 #scaling for observation time #aLIGO assumes 2 years of data

In [ ]:
time5y = 5*3.154e+7

In [ ]:
#plt.loglog(et_curve['freq'], Sn2OmGW(et_curve['freq'], et_curve['ETDHF']), label="ET-D HF", linestyle="--")
#plt.loglog(et_curve['freq'], Sn2OmGW(et_curve['freq'], et_curve['ETDLF']), label="ET-D LF", linestyle = '-.')
#plt.loglog(et_curve['freq'], Sn2OmGW(et_curve['freq'], et_curve['Sum']), label="ET-D sum")

#plt.loglog(et_curve['freq'], Sn2OmGW_wT(et_curve['freq'], et_curve['ETDHF'], time), label="ET-D HF", linestyle="--")
#plt.loglog(et_curve['freq'], Sn2OmGW_wT(et_curve['freq'], et_curve['ETDLF'], time), label="ET-D LF", linestyle = '-.')
plt.loglog(et_curve['freq'], Sn2OmGW_wT(et_curve['freq'], et_curve['Sum'], time2y), label="ET-D sum, 2 yrs")
plt.loglog(et_curve['freq'], Sn2OmGW_wT(et_curve['freq'], et_curve['Sum'], time5y), label="ET-D sum, 5 yrs")



plt.grid(True)
plt.ylabel("$\Omega_{GW}(f)$")
plt.xlabel("frequency (Hz)")
plt.legend()
plt.tight_layout()
#plt.savefig("ET_om_gw_2yr_integration.pdf")
plt.show()

In [ ]:
nano_curve = np.genfromtxt("nanograv18_omega.txt", names=('nanoFreq', 'NANOGrav18'))

In [ ]:
nano_curve = Table(nano_curve)

In [ ]:
nano_hc = np.genfromtxt("nanograv18_hc.txt", names=('nanoFreq', 'NANOGrav18'))

In [ ]:
nano_hc = Table(nano_hc)

In [ ]:
plt.loglog(nano_hc['nanoFreq'], nano_hc['NANOGrav18'], label="NANOGrav 2018")

In [ ]:
aligo_dat = np.genfromtxt("aLIGO_design.dat", delimiter='', skip_header=1, names=("freq", "omega"))

In [ ]:
aligo_dat = Table(aligo_dat)

In [ ]:
plt.loglog(aligo_dat['freq'], aligo_dat['omega'])

In [ ]:
lisa_dat = np.genfromtxt("PICurve_LISA.dat", delimiter='', skip_header=1, names=("freq", "omega"))

In [ ]:
lisa_dat = Table(lisa_dat)

In [ ]:
plt.loglog(nano_curve['nanoFreq'], nano_curve['NANOGrav18'], label="NANOGrav 2018")

In [ ]:
nanoOmega11 = 3.4e-10/(.68)**2 #NG limit was in h=1, need to divide by h=.68^2
freq_yr = 1/3.154e+7
n = 150 # remove the first n points for ET so that the high frequency cutoff does not dominate the figure

In [ ]:
#plt.loglog(et_curve['freq'], Sh2OmGW(et_curve['freq'], et_curve['ETDHF'])/time, label="ET-D HF", linestyle="--")
#plt.loglog(et_curve['freq'], Sh2OmGW(et_curve['freq'], et_curve['ETDLF'])/time, label="ET-D LF", linestyle = '-.')
plt.loglog(nano_curve['nanoFreq'], nano_curve['NANOGrav18']/(.68)**2, label="NANOGrav 2018")
plt.loglog(aligo_dat['freq'], aligo_dat['omega'], label= "aLIGO design")
plt.loglog(et_curve['freq'][n:], Sn2OmGW_wT(et_curve['freq'][n:], et_curve['Sum'][n:], time2y), label="ET-D", ls = 'dashed')
#plt.loglog(et_curve['freq'][n:], Sn2OmGW_wT(et_curve['freq'][n:], et_curve['Sum'][n:], time5y), label="ET-D 5 yrs")

plt.loglog(ce_curve['freq'][n:], Sn2OmGW_wT(ce_curve['freq'][n:], ce_curve['CE'][n:], time2y), label="Cosmic Explorer", ls = 'dashed')
#plt.loglog(ce_curve['freq'], Sn2OmGW_wT(ce_curve['freq'], ce_curve['CE'], time5y), label="CE, 5 yrs")



plt.loglog(lisa_dat['freq'], lisa_dat['omega'], label= "LISA", ls = 'dashed')
#plt.annotate('NG11',(freq_yr, nanoOmega11))

plt.grid(True)
plt.ylabel("$\Omega_{GW}(f)$")
plt.xlabel("Frequency (Hz)")
plt.legend(loc=9)
#plt.ylim(1e-16, 1e-2)
#plt.xlim(1e-9, 1e4)
plt.tight_layout()
#plt.savefig("all_exper_w2yrs.pdf")
plt.show()

In [ ]:
# plotting preamble

fig_width_pt = 245.27 #513.17           # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

params = {'backend': 'pdf',
        'axes.labelsize': 7,
        'lines.markersize': 2,
        'font.size': 7,
        'xtick.major.size':6,
        'xtick.minor.size':3,  
        'ytick.major.size':6,
        'ytick.minor.size':3, 
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.0,
        'legend.fontsize': 7,
        'xtick.labelsize': 7,
        'ytick.labelsize': 7,
        'savefig.dpi':200,
        'path.simplify':True,
        #'font.family': 'serif',
        'font.serif':'Times',
        'text.latex.preamble': [r'\usepackage{amsmath}'],
        'text.usetex':True,
        #'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
        #                        'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
        #'font.serif':cm,
        'figure.figsize': fig_size}
plt.rcParams.update(params)
